In [1]:
import ray
import numpy as np

from markets.realistic import RayMarketScenario, MarketMaker, USITMarket, ChartInvestor
from markets.realistic import INTRINSIC_VALUE, BiasedMarketView
from markets.realistic.Clock import Clock

---

In [2]:
ray.init()
np.random.seed(17)
clock = Clock(n_seconds=60, n_minutes=60, n_hours=24, n_days=256)
# sc = SynchronousMarketScenario(clock)
sc = RayMarketScenario(clock)

---

In [3]:
market = USITMarket({'TSMC': 186.73, 'AAPL': 201.22, 'TSLA': 462.4},noise=0.0)
mm = MarketMaker(market)
market_makers = sc.register_market_makers(mm)

In [4]:
mm = market_makers[0]
mm.osid()

'6294 - 20499767200'

---

In [5]:
noise = .05
biases = {INTRINSIC_VALUE: lambda v: 0.96 * v}
biased_market_view = BiasedMarketView(market, biases)
warren = ChartInvestor(market=biased_market_view,
                       name='Warren Buffet',
                       portfolio={'TSMC': 5000},
                       cash=200_000)

# Michael has a bias as to overestimate the intrinsic value (OK, I know he wouldn't - ever!)
biases = {INTRINSIC_VALUE: lambda v: 1.04 * v}
#biases = {INTRINSIC_VALUE: lambda v: np.random.normal(1.03 * v, v * noise)}
biased_market_view = BiasedMarketView(market, biases)
michael = ChartInvestor(market=biased_market_view,
                        name='Michael Burry',
                        portfolio={'TSMC': 5000},
                        cash=200_000)

In [6]:
investors = sc.register_investors(warren, michael)

---

In [7]:
warren, michael = investors
warren.osid(), michael.osid()

('6285 - 20405268544', '6286 - 4494577728')

In [8]:
mm.get_prices()

{'TSMC': {'bid': None, 'ask': None, 'last': 188.60194476307495},
 'AAPL': {'bid': None, 'ask': None, 'last': 203.2372051905208},
 'TSLA': {'bid': None, 'ask': None, 'last': 467.0355018392645}}

---

In [9]:
sc.tick(seconds=10)


# TODO: Introduce simple Momentum Investors
# TODO: Investors may create market orders (Retail)
# TODO: Introduce node- and stock-specific market makers
# TODO: Handle order expiry at market maker and investor side
# TODO: Introduce stop loss strategy for investors
# TODO: Introduce central logging (return values from 'tick')
# TODO: Introduce reporting from the market maker



In [ ]:
mm.get_prices()

In [ ]:
michael.osid(), warren.osid()

In [10]:
warren.get_portfolio()

{'TSMC': 4895.914885738468, 'CASH': 219565.09971434614}

In [ ]:
sc.investors[0].get_portfolio()

In [ ]:
type(warren)

In [11]:
mm.get_prices()

{'TSMC': {'bid': 188.19, 'ask': 190.28, 'last': 190.28},
 'AAPL': {'bid': None, 'ask': None, 'last': 203.2372051905208},
 'TSLA': {'bid': None, 'ask': None, 'last': 467.0355018392645}}

---
---

In [ ]:
sc.investors[0].tick(Clock())

In [ ]:
sc.investors[0].delegate

In [ ]:
ray.get(sc.investors[0].delegate.identify.remote())

In [ ]:
warren.get_portfolio()

In [ ]:
ray.shutdown()